In [1]:
import wikipedia
import ipywidgets
import re
import textwrap
import urllib
import pypandoc
from IPython.display import display,Markdown

In [2]:
level = ipywidgets.IntSlider(
    value=4,
    min=1,
    max=5,
    step=1,
    description='Difficulty')

level

IntSlider(value=4, description='Difficulty', max=5, min=1)

In [11]:
urlname = f'https://randomincategory.toolforge.org/?category=All%20Wikipedia%20level-{level.value}%20vital%20articles&server=en.wikipedia.org&cmnamespace=&cmtype=&returntype=subject'

try:
  name = urllib.request.urlopen(urlname).read(500).decode().split('title>')[1].split(' - Wikipedia')[0]
  content = wikipedia.page(name,auto_suggest=False).content
except:
  name = urllib.request.urlopen(urlname).read(500).decode().split('title>')[1].split(' - Wikipedia')[0]
  content = wikipedia.page(name,auto_suggest=False).content

def print_redacted(new_string,count_to_word_dict):
  string_to_print = ''
  delims=['\n',r'===',r'==',r'.',r',',r'(',r')',r'[',r']',r'<',r'>',r'"',r':',r';',r'-',r'/',r'–',r'=',r'%']
  for word in re.split(r'([ |\n|=|.|,|(|)|[|]|<|>|"|:|;|-|/|–|%])',new_string):
    if type(word)==str:
      if word in delims:
        string_to_print += word
      elif word in count_to_word_dict.keys():
        if len(count_to_word_dict[word])>0:
          string_to_print += count_to_word_dict[word][0]+' '
      else:
        pass
  string_to_print = pypandoc.convert_text(string_to_print,'markdown_strict',format='mediawiki',extra_args=["--shift-heading-level-by=1","--atx-headers"])
  string_to_print = re.sub('(?<=[A-z])(\n)(?=[A-z])',' ',string_to_print) # remove random extra newlines
  string_to_print = re.sub('(#{3,5} )(?P<Header>.*)(?=\n\n)','**\g<Header>** ',string_to_print) #make all headers just bold
  display(Markdown(string_to_print))

word_to_count_dict = {}
count_to_word_dict = {}
count_the_word_dict = {}
new_string =''
count = 9897098660698
delims=['\n',r'.',r',',r'(',r')',r'[',r']',r'<',r'>',r'"',r':',r';',r'-',r'/',r'–',r'=',r'%',r'',r' ']
words = [x for x in re.split(r'([ |\n|=|.|,|(|)|[|]|<|>|"|:|;|-|/|–|%])','### '+name+' \n\n '+content)]
for word in words:
  if type(word)==str:
    word = word.lower()
    if word in delims:
      if word not in ['',' ']:
        new_string += word
    elif word not in word_to_count_dict.keys():
      word_to_count_dict[word]=str(count)
      count_to_word_dict[str(count)]=['-'*len(word),word]
      new_string += str(count)+' '
      count += 1
      count_the_word_dict[word]=1
    else:
      new_string += word_to_count_dict[word]+' '
      count_the_word_dict[word]+=1

#new_string = textwrap.fill(new_string,break_long_words=False,width=200,replace_whitespace=False,drop_whitespace=False, break_on_hyphens=False)
tried = []
name_list = re.split(r'[ |\n|===|==|.|,|(|)|[|]|<|>|"|:|;|-|/|–|=|%]{1,3}',name)
name_list = [x.lower() for x in name_list]

starting_guesses = ['a','an','is','the','of', 'in', 'to', 'was', 'for', 'as',
'around', 'from', 'since', 'with', 'and', 'or', 'on', 'along', 'through', 'amid', 
'into', 'like', 'by', 'between', 'than', 'about', 'until', 'at', 'but', 'among', 
'following', 'over', 'after', 'during', 'behind', 'up', 'down', 'before', 'onto', 
'without', 'unlike', 'because', 'against', 'inside', 'outside','except','per']
for eachguess in starting_guesses:
  try:
    tried.append(eachguess)
    count = word_to_count_dict[eachguess]
    count_to_word_dict[count]=count_to_word_dict[count][1:]
    name_list.remove(eachguess)
  except:
    pass

In [14]:
guess = ipywidgets.Text(value='the')

guess

Text(value='the')

In [15]:
thisguess = guess.value.lower()
if thisguess in tried:
  display(Markdown(('already tried, here is what you tried already:'+textwrap.fill(', '.join(tried)))))
  print_redacted(new_string,count_to_word_dict)
elif thisguess not in word_to_count_dict.keys():
  display(Markdown('Nope'))
  print_redacted(new_string,count_to_word_dict)
  tried.append(thisguess)
else:
  tried.append(thisguess)
  if thisguess in name_list:
    name_list.remove(thisguess)
  if len(name_list)>0:
    count = word_to_count_dict[thisguess]
    count_to_word_dict[count]=count_to_word_dict[count][1:]
    display(Markdown(f"{thisguess} appears {count_the_word_dict[thisguess]} times"))
  else:
    display(Markdown('Hooray! \n'))
    for k,v in count_to_word_dict.items():
      if len(v)>1:
        count_to_word_dict[k]=v[1:]
  print_redacted(new_string,count_to_word_dict)

Hooray! 


cavalry

historically ,cavalry (from the french word cavalerie ,itself derived from "cheval "meaning "horse ")are soldiers or warriors who fight mounted on horseback .cavalry were the most mobile of the combat arms
,operating as light cavalry in the roles of reconnaissance ,screening
,and skirmishing in many armies ,or as heavy cavalry for decisive shock attacks in other armies .an individual soldier in the cavalry is known by a number of designations depending on era and tactics ,such as cavalryman ,horseman ,trooper ,cataphract ,knight ,hussar ,uhlan ,mamluk
,cuirassier ,lancer ,dragoon ,or horse archer .the designation of cavalry was not usually given to any military forces that used other animals for mounts ,such as camels or elephants .infantry who moved on horseback ,but dismounted to fight on foot ,were known in the early 17th to the early 18th century as dragoons ,a class of mounted infantry which in most armies later evolved into standard cavalry while retaining their historic designation . cavalry had the advantage of improved mobility
,and a soldier fighting from horseback also had the advantages of greater height ,speed ,and inertial mass over an opponent on foot
.another element of horse mounted warfare is the psychological impact a mounted soldier can inflict on an opponent . the speed ,mobility ,and shock value of cavalry was greatly appreciated and exploited in armed forces in the ancient and middle ages ;some forces were mostly cavalry
,particularly in nomadic societies of asia ,notably the huns of attila and the later mongol armies .in europe ,cavalry became increasingly armoured (heavy ),and eventually evolving into the mounted knights of the medieval period .during the 17th century ,cavalry in europe discarded most of its armor ,which was ineffective against the muskets and cannons that were coming into common use ,and by the mid -18th century armor had mainly fallen into obsolescence ,although some regiments retained a small thickened cuirass that offered protection against lances ,sabres ,and bayonets ;including some protection against a shot from distance . in the interwar period ,while some cavalry still served during world war ii (notably in the red army ,the mongolian people's army ,the royal italian army ,the romanian army ,the polish land forces ,and light reconnaissance units within the waffen ss )many cavalry units were converted into motorized infantry and mechanized infantry units ,or reformed as tank troops .the cavalry tank or cruiser tank was one designed with a speed and purpose beyond that of infantry tanks and would subsequently develop into the main battle tank . most cavalry units that are horse -mounted in modern armies serve in purely ceremonial roles ,or as mounted infantry in difficult terrain such as mountains or heavily forested areas .modern usage of the term generally refers to units performing the role of reconnaissance ,surveillance ,and target acquisition (analogous to historical light cavalry )or main battle tank units (analogous to historical heavy cavalry ).

**role** 

historically ,cavalry was divided into light cavalry and heavy cavalry
.the differences were their roles in combat ,the size of their mounts
,and how much armor was worn by the mount and rider . heavy cavalry
,such as byzantine cataphracts and knights of the early middle ages in europe ,were used as shock troops ,charging the main body of the enemy at the height of a battle ;in many cases their actions decided the outcome of the battle ,hence the later term battle cavalry .light cavalry ,such as horse archers ,hussars ,and cossack cavalry ,were assigned all the numerous roles that were ill -suited to more narrowly
-focused heavy forces .this includes scouting ,deterring enemy scouts
,foraging ,raiding ,skirmishing ,pursuit of retreating enemy forces
,screening of retreating friendly forces ,linking separated friendly forces ,and countering enemy light forces in all these same roles .
light and heavy cavalry roles continued through early modern warfare
,but armor was reduced ,with light cavalry mostly unarmored .yet many cavalry units still retained cuirasses and helmets for their protective value against sword and bayonet strikes ,and the morale boost these provide to the wearers ,despite these giving little protection from firearms .by this time the main difference between light and heavy cavalry was their training ;the former was regarded as best suited for harassment and reconnaissance ,while the latter was considered best for close -order charges .by the start of the 20th century ,as total battlefield firepower increased ,all cavalry tended to become dragoons in practice ,riding mounted between battles ,but dismounting to act as infantry during any battle ,even if many retained their unit names that reflected their older cavalry roles . with the development of armored warfare ,the heavy cavalry role of decisive shock troops had been taken over by armored units employing medium and heavy tanks ,and later main battle tanks .despite horse -born cavalry becoming obsolete ,the term cavalry is still used ,referring in modern times to units continuing to fulfill the traditional light cavalry roles ,employing fast armored cars
.light tanks ,and infantry fighting vehicles instead of horses ,while air cavalry employs helicopters .

**early history** 

**origins** 

before the iron age ,the role of cavalry on the battlefield was largely performed by light chariots .the chariot originated with the sintashta
-petrovka culture in central asia and spread by nomadic or semi -nomadic indo -iranians .the chariot was quickly adopted by settled peoples both as a military technology and an object of ceremonial status ,especially by the pharaohs of the new kingdom of egypt from 1550 bc as well as the assyrian army and babylonian royalty .the power of mobility given by mounted units was recognized early on ,but was offset by the difficulty of raising large forces and by the inability of horses (then mostly small )to carry heavy armor .nonetheless ,there are indications that
,from the 15th century bc onwards ,horseback riding was practiced amongst the military elites of the great states of the ancient near east
,most notably those in egypt ,assyria ,the hittite empire ,and mycenaean greece .cavalry techniques ,and the rise of true cavalry ,were an innovation of equestrian nomads of the central asian and iranian steppe and pastoralist tribes such as the iranic parthians and sarmatians .

the photograph above left shows assyrian cavalry from reliefs of 865
–860 bc .at this time ,the men had no spurs ,saddles ,saddle cloths ,or stirrups .fighting from the back of a horse was much more difficult than mere riding .the cavalry acted in pairs ;the reins of the mounted archer were controlled by his neighbour's hand .even at this early time
,cavalry used swords ,shields ,spears ,and bows .the sculpture implies two types of cavalry ,but this might be a simplification by the artist
.later images of assyrian cavalry show saddle cloths as primitive saddles ,allowing each archer to control his own horse .as early as 490
bc a breed of large horses was bred in the nisaean plain in media to carry men with increasing amounts of armour (herodotus 7 ,40 & 9 ,20
),but large horses were still very exceptional at this time .by the fourth century bc the chinese during the warring states period (403 –221
bc )began to use cavalry against rival states ,and by 331 bc when alexander the great defeated the persians the use of chariots in battle was obsolete in most nations ;despite a few ineffective attempts to revive scythed chariots .the last recorded use of chariots as a shock force in continental europe was during the battle of telamon in 225 bc .
however ,chariots remained in use for ceremonial purposes such as carrying the victorious general in a roman triumph ,or for racing .
outside of mainland europe ,the southern britons met julius caesar with chariots in 55 and 54 bc ,but by the time of the roman conquest of britain a century later chariots were obsolete ,even in britannia .the last mention of chariot use in britain was by the caledonians at the mons graupius ,in 84 ad .

**ancient greece :city -states ,thebes ,thessaly and macedonia** 

during the classical greek period cavalry were usually limited to those citizens who could afford expensive war -horses .three types of cavalry became common :light cavalry ,whose riders ,armed with javelins ,could harass and skirmish ;heavy cavalry ,whose troopers ,using lances ,had the ability to close in on their opponents ;and finally those whose equipment allowed them to fight either on horseback or foot .the role of horsemen did however remain secondary to that of the hoplites or heavy infantry who comprised the main strength of the citizen levies of the various city states .cavalry played a relatively minor role in ancient greek city -states ,with conflicts decided by massed armored infantry
.however ,thebes produced pelopidas ,their first great cavalry commander
,whose tactics and skills were absorbed by philip ii of macedon when philip was a guest -hostage in thebes .thessaly was widely known for producing competent cavalrymen ,and later experiences in wars both with and against the persians taught the greeks the value of cavalry in skirmishing and pursuit .the athenian author and soldier xenophon in particular advocated the creation of a small but well -trained cavalry force ;to that end ,he wrote several manuals on horsemanship and cavalry operations .the macedonian kingdom in the north ,on the other hand
,developed a strong cavalry force that culminated in the hetairoi
(companion cavalry )of philip ii of macedon and alexander the great .in addition to these heavy cavalry ,the macedonian army also employed lighter horsemen called prodromoi for scouting and screening ,as well as the macedonian pike phalanx and various kinds of light infantry .there were also the ippiko (or "horserider "),greek "heavy "cavalry ,armed with kontos (or cavalry lance ),and sword .these wore leather armour or mail plus a helmet .they were medium rather than heavy cavalry ,meaning that they were better suited to be scouts ,skirmishers ,and pursuers rather than front line fighters .the effectiveness of this combination of cavalry and infantry helped to break enemy lines and was most dramatically demonstrated in alexander's conquests of persia ,bactria
,and northwestern india .

**roman republic and early empire** 

the cavalry in the early roman republic remained the preserve of the wealthy landed class known as the equites—men who could afford the expense of maintaining a horse in addition to arms and armor heavier than those of the common legions .horses were provided by the republic and could be withdrawn if neglected or misused ,together with the status of being a cavalryman .as the class grew to be more of a social elite instead of a functional property -based military grouping ,the romans began to employ italian socii for filling the ranks of their cavalry
.the weakness of roman cavalry was demonstrated by hannibal barca during the second punic war where he used his superior mounted forces to win several battles .the most notable of these was the battle of cannae
,where he inflicted a catastrophic defeat on the romans .at about the same time the romans began to recruit foreign auxiliary cavalry from among gauls ,iberians ,and numidians ,the last being highly valued as mounted skirmishers and scouts (see numidian cavalry ).julius caesar had a high opinion of his escort of germanic mixed cavalry ,giving rise to the cohortes equitatae .early emperors maintained an ala of batavian cavalry as their personal bodyguards until the unit was dismissed by galba after the batavian rebellion .for the most part ,roman cavalry during the early republic functioned as an adjunct to the legionary infantry and formed only one -fifth of the standing force comprising a consular army .except in times of major mobilisation about 1 ,800
horsemen were maintained ,with three hundred attached to each legion .
the relatively low ratio of horsemen to infantry does not mean that the utility of cavalry should be underestimated ,as its strategic role in scouting ,skirmishing ,and outpost duties was crucial to the romans'
capability to conduct operations over long distances in hostile or unfamiliar territory .on some occasions roman cavalry also proved its ability to strike a decisive tactical blow against a weakened or unprepared enemy ,such as the final charge at the battle of aquilonia
.after defeats such as the battle of carrhae ,the romans learned the importance of large cavalry formations from the parthians . at the same time heavy spears and shields modelled on those favoured by the horsemen of the greek city -states were adopted to replace the lighter weaponry of early rome . these improvements in tactics and equipment reflected those of a thousand years earlier when the first iranians to reach the iranian plateau forced the assyrians to undertake similar reform
.nonetheless ,the romans would continue to rely mainly on their heavy infantry supported by auxiliary cavalry .

**late roman empire and the migration period** 

in the army of the late roman empire ,cavalry played an increasingly important role .the spatha ,the classical sword throughout most of the
1st millennium was adopted as the standard model for the empire's cavalry forces .by the 6th century these had evolved into lengthy straight weapons influenced by persian and other eastern patterns .the most widespread employment of heavy cavalry at this time was found in the forces of the iranian empires ,the parthians and their persian sasanian successors .both ,but especially the former ,were famed for the cataphract (fully armored cavalry armed with lances )even though the majority of their forces consisted of lighter horse archers .the west first encountered this eastern heavy cavalry during the hellenistic period with further intensive contacts during the eight centuries of the roman –persian wars .at first the parthians' mobility greatly confounded the romans ,whose armoured close -order infantry proved unable to match the speed of the parthians .however ,later the romans would successfully adapt such heavy armor and cavalry tactics by creating their own units of cataphracts and clibanarii .the decline of the roman infrastructure made it more difficult to field large infantry forces ,and during the
4th and 5th centuries cavalry began to take a more dominant role on the european battlefield ,also in part made possible by the appearance of new ,larger breeds of horses .the replacement of the roman saddle by variants on the scythian model ,with pommel and cantle ,was also a significant factor as was the adoption of stirrups and the concomitant increase in stability of the rider's seat .armored cataphracts began to be deployed in eastern europe and the near east ,following the precedents established by persian forces ,as the main striking force of the armies in contrast to the earlier roles of cavalry as scouts
,raiders ,and outflankers .the late -roman cavalry tradition of organized units in a standing army differed fundamentally from the nobility of the germanic invaders—individual warriors who could afford to provide their own horses and equipment .while there was no direct linkage with these predecessors the early medieval knight also developed as a member of a social and martial elite ,able to meet the considerable expenses required by his role from grants of land and other incomes .

**asia** 

**central asia** 

xiongnu ,tujue ,avars ,kipchaks ,khitans ,mongols ,don cossacks and the various turkic peoples are also examples of the horse -mounted groups that managed to gain substantial successes in military conflicts with settled agrarian and urban societies ,due to their strategic and tactical mobility .as european states began to assume the character of bureaucratic nation -states supporting professional standing armies
,recruitment of these mounted warriors was undertaken in order to fill the strategic roles of scouts and raiders .

the best known instance of the continued employment of mounted tribal auxiliaries were the cossack cavalry regiments of the russian empire .in eastern europe ,and out onto the steppes ,cavalry remained important much longer and dominated the scene of warfare until the early 17th century and even beyond ,as the strategic mobility of cavalry was crucial for the semi -nomadic pastoralist lives that many steppe cultures led .tibetans also had a tradition of cavalry warfare ,in several military engagements with the chinese tang dynasty (618 –907 ad
).

**khanates of central asia** 

**east asia** 

**china** 

further east ,the military history of china ,specifically northern china
,held a long tradition of intense military exchange between han chinese infantry forces of the settled dynastic empires and the mounted nomads or "barbarians "of the north .the naval history of china was centered more to the south ,where mountains ,rivers ,and large lakes necessitated the employment of a large and well -kept navy . in 307 bc ,king wuling of zhao ,the ruler of the former state of jin ,ordered his commanders and troops to adopt the trousers of the nomads as well as practice the nomads' form of mounted archery to hone their new cavalry skills .

the adoption of massed cavalry in china also broke the tradition of the chariot -riding chinese aristocracy in battle ,which had been in use since the ancient shang dynasty (c 1600 –1050 bc ).by this time large chinese infantry -based armies of 100 ,000 to 200 ,000 troops were now buttressed with several hundred thousand mounted cavalry in support or as an effective striking force .the handheld pistol -and -trigger crossbow was invented in china in the fourth century bc ;it was written by the song dynasty scholars zeng gongliang ,ding du ,and yang weide in their book wujing zongyao (1044 ad )that massed missile fire by crossbowmen was the most effective defense against enemy cavalry charges
.

on many occasions the chinese studied nomadic cavalry tactics and applied the lessons in creating their own potent cavalry forces ,while in others they simply recruited the tribal horsemen wholesale into their armies ;and in yet other cases nomadic empires proved eager to enlist chinese infantry and engineering ,as in the case of the mongol empire and its sinicized part ,the yuan dynasty (1279 –1368 ).the chinese recognized early on during the han dynasty (202 bc –220 ad )that they were at a disadvantage in lacking the number of horses the northern nomadic peoples mustered in their armies .emperor wu of han (r 141 –87
bc )went to war with the dayuan for this reason ,since the dayuan were hoarding a massive amount of tall ,strong ,central asian bred horses in the hellenized –greek region of fergana (established slightly earlier by alexander the great ).although experiencing some defeats early on in the campaign ,emperor wu's war from 104 bc to 102 bc succeeded in gathering the prized tribute of horses from fergana . cavalry tactics in china were enhanced by the invention of the saddle -attached stirrup by at least the 4th century ,as the oldest reliable depiction of a rider with paired stirrups was found in a jin dynasty tomb of the year 322 ad .the chinese invention of the horse collar by the 5th century was also a great improvement from the breast harness ,allowing the horse to haul greater weight without heavy burden on its skeletal structure .

**korea** 

the horse warfare of korea was first started during the ancient korean kingdom gojoseon .since at least the 3rd century bc ,there was influence of northern nomadic peoples and yemaek peoples on korean warfare .by roughly the first century bc ,the ancient kingdom of buyeo also had mounted warriors .the cavalry of goguryeo ,one of the three kingdoms of korea ,were called gaemamusa (개마무사 ,鎧馬武士 ),and were renowned as a fearsome heavy cavalry force .king gwanggaeto the great often led expeditions into the baekje ,gaya confederacy ,buyeo ,later yan and against japanese invaders with his cavalry . in the 12th century
,jurchen tribes began to violate the goryeo –jurchen borders ,and eventually invaded goryeo korea .after experiencing the invasion by the jurchen ,korean general yun gwan realized that goryeo lacked efficient cavalry units .he reorganized the goryeo military into a professional army that would contain decent and well -trained cavalry units .in 1107
,the jurchen were ultimately defeated ,and surrendered to yun gwan .to mark the victory ,general yun built nine fortresses to the northeast of the goryeo –jurchen borders (동북 9성 ,東北 九城 ).

**japan** 

the ancient japanese of the kofun period also adopted cavalry and equine culture by the 5th century ad .the emergence of the samurai aristocracy led to the development of armoured horse archers ,themselves to develop into charging lancer cavalry as gunpowder weapons rendered bows obsolete
.japanese cavalry was largely made up of landowners who would be upon a horse to better survey the troops they were called upon to bring to an engagement ,rather than traditional mounted warfare seen in other cultures with massed cavalry units . an example is yabusame (流鏑馬? ),a type of mounted archery in traditional japanese archery .an archer on a running horse shoots three special "turnip -headed "arrows successively at three wooden targets . this style of archery has its origins at the beginning of the kamakura period .minamoto no yoritomo became alarmed at the lack of archery skills his samurai had .he organized yabusame as a form of practice . currently ,the best places to see yabusame performed are at the tsurugaoka hachiman -gū in kamakura and shimogamo shrine in kyoto (during aoi matsuri in early may ).it is also performed in samukawa and on the beach at zushi ,as well as other locations .
kasagake or kasakake (笠懸 ,かさがけ lit ."hat shooting ")is a type of japanese mounted archery .in contrast to yabusame ,the types of targets are various and the archer shoots without stopping the horse .while yabusame has been played as a part of formal ceremonies ,kasagake has developed as a game or practice of martial arts ,focusing on technical elements of horse archery .

**south asia** 

**indian subcontinent** 

in the indian subcontinent ,cavalry played a major role from the gupta dynasty (320 –600 )period onwards .india has also the oldest evidence for the introduction of toe -stirrups .indian literature contains numerous references to the mounted warriors of the central asian horse nomads ,notably the sakas ,kambojas ,yavanas ,pahlavas and paradas
.numerous puranic texts refer to a conflict in ancient india (16th century bc )in which the horsemen of five nations ,called the "five hordes "(pañca .ganan )or kṣatriya hordes (kṣatriya ganah ),attacked and captured the state of ayudhya by dethroning its vedic king bahu

the mahabharata ,ramayana ,numerous puranas and some foreign sources attest that the kamboja cavalry frequently played role in ancient wars
.v .r .ramachandra dikshitar writes :"both the puranas and the epics agree that the horses of the sindhu and kamboja regions were of the finest breed ,and that the services of the kambojas as cavalry troopers were utilised in ancient wars ".j .a .o .s .writes :"most famous horses are said to come either from sindhu or kamboja ;of the latter (i .e .the kamboja ),the indian epic mahabharata speaks among the finest horsemen
".

the mahabharata speaks of the esteemed cavalry of the kambojas ,sakas
,yavanas and tusharas ,all of whom had participated in the kurukshetra war under the supreme command of kamboja ruler sudakshin kamboj
.mahabharata and vishnudharmottara purana pay especial attention to the kambojas ,yavansa ,gandharas etc .being ashva .yuddha .kushalah (expert cavalrymen ).in the mahabharata war ,the kamboja cavalry along with that of the sakas ,yavanas is reported to have been enlisted by the kuru king duryodhana of hastinapura .herodotus (c 484 –c 425 bc )attests that the gandarian mercenaries (i .e .gandharans /kambojans of gandari strapy of achaemenids )from the 20th strapy of the achaemenids were recruited in the army of emperor xerxes i (486 –465 bc ),which he led against the hellas .similarly ,the men of the mountain land from north of kabul
-river equivalent to medieval kohistan (pakistan ),figure in the army of darius iii against alexander at arbela ,providing a cavalry force and 15
elephants .this obviously refers to kamboja cavalry south of hindukush .
the kambojas were famous for their horses ,as well as cavalrymen (asva
-yuddha -kushalah ).on account of their supreme position in horse (ashva
)culture ,they were also popularly known as ashvakas ,i .e .the
"horsemen "and their land was known as "home of horses ".they are the assakenoi and aspasioi of the classical writings ,and the ashvakayanas and ashvayanas in pāṇini's ashtadhyayi .the assakenoi had faced alexander with 30 ,000 infantry ,20 ,000 cavalry and 30 war elephants
.scholars have identified the assakenoi and aspasioi clans of kunar and swat valleys as a section of the kambojas .these hardy tribes had offered stubborn resistance to alexander (c 326 bc )during latter's campaign of the kabul ,kunar and swat valleys and had even extracted the praise of the alexander's historians .these highlanders ,designated as
"parvatiya ayudhajivinah "in pāṇini's astadhyayi ,were rebellious
,fiercely independent and freedom -loving cavalrymen who never easily yielded to any overlord .the sanskrit drama mudra -rakashas by visakha dutta and the jaina work parishishtaparvan refer to chandragupta's (c
320 bc –c 298 bc )alliance with himalayan king parvataka .the himalayan alliance gave chandragupta a formidable composite army made up of the cavalry forces of the shakas ,yavanas ,kambojas ,kiratas ,parasikas and bahlikas as attested by mudra -rakashas (mudra -rakshasa 2 ).these hordes had helped chandragupta maurya defeat the ruler of magadha and placed chandragupta on the throne ,thus laying the foundations of mauryan dynasty in northern india . the cavalry of hunas and the kambojas is also attested in the raghu vamsa epic poem of sanskrit poet kalidasa .raghu of kalidasa is believed to be chandragupta ii
(vikaramaditya )(375 –413 /15 ad ),of the well -known gupta dynasty . as late as the mediaeval era ,the kamboja cavalry had also formed part of the gurjara -pratihara armed forces from the eighth to the 10th centuries ad .they had come to bengal with the pratiharas when the latter conquered part of the province .ancient kambojas organised military sanghas and shrenis (corporations )to manage their political and military affairs ,as arthashastra of kautiliya as well as the mahabharata record .they are described as ayuddha -jivi or shastr
-opajivis (nations -in -arms ),which also means that the kamboja cavalry offered its military services to other nations as well .there are numerous references to kambojas having been requisitioned as cavalry troopers in ancient wars by outside nations .

**mughal empire** 

the mughal armies (lashkar )were primarily a cavalry force .the elite corps were the ahadi who provided direct service to the emperor and acted as guard cavalry .supplementary cavalry or dakhilis were recruited
,equipped and paid by the central state .this was in contrast to the tabinan horsemen who were the followers of individual noblemen .their training and equipment varied widely but they made up the backbone of the mughal cavalry .finally there were tribal irregulars led by and loyal to tributary chiefs .these included hindus ,afghans and turks summoned for military service when their autonomous leaders were called on by the imperial government .

**european middle ages** 

as the quality and availability of heavy infantry declined in europe with the fall of the roman empire ,heavy cavalry became more effective
.infantry that lack the cohesion and discipline of tight formations are more susceptible to being broken and scattered by shock combat—the main role of heavy cavalry ,which rose to become the dominant force on the european battlefield .as heavy cavalry increased in importance ,it became the main focus of military development .the arms and armour for heavy cavalry increased ,the high -backed saddle developed ,and stirrups and spurs were added ,increasing the advantage of heavy cavalry even more .this shift in military importance was reflected in society as well
;knights took centre stage both on and off the battlefield .these are considered the "ultimate "in heavy cavalry :well -equipped with the best weapons ,state -of -the -art armour from head to foot ,leading with the lance in battle in a full -gallop ,close -formation "knightly charge
"that might prove irresistible ,winning the battle almost as soon as it begun .

but knights remained the minority of total available combat forces ;the expense of arms ,armour ,and horses was only affordable to a select few
.while mounted men -at -arms focused on a narrow combat role of shock combat ,medieval armies relied on a large variety of foot troops to fulfill all the rest (skirmishing ,flank guards ,scouting ,holding ground ,etc .).medieval chroniclers tended to pay undue attention to the knights at the expense of the common soldiers ,which led early students of military history to suppose that heavy cavalry was the only force that mattered on medieval european battlefields .but well -trained and disciplined infantry could defeat knights . massed english longbowmen triumphed over french cavalry at crécy ,poitiers and agincourt ,while at gisors (1188 ),bannockburn (1314 ),and laupen (1339 ),foot -soldiers proved they could resist cavalry charges as long as they held their formation .once the swiss developed their pike squares for offensive as well as defensive use ,infantry started to become the principal arm
.this aggressive new doctrine gave the swiss victory over a range of adversaries ,and their enemies found that the only reliable way to defeat them was by the use of an even more comprehensive combined arms doctrine ,as evidenced in the battle of marignano .the introduction of missile weapons that required less skill than the longbow ,such as the crossbow and hand cannon ,also helped remove the focus somewhat from cavalry elites to masses of cheap infantry equipped with easy -to -learn weapons .these missile weapons were very successfully used in the hussite wars ,in combination with wagenburg tactics . this gradual rise in the dominance of infantry led to the adoption of dismounted tactics
.from the earliest times knights and mounted men -at -arms had frequently dismounted to handle enemies they could not overcome on horseback ,such as in the battle of the dyle (891 )and the battle of bremule (1119 ),but after the 1350s this trend became more marked with the dismounted men -at -arms fighting as super -heavy infantry with two
-handed swords and poleaxes .in any case ,warfare in the middle ages tended to be dominated by raids and sieges rather than pitched battles
,and mounted men -at -arms rarely had any choice other than dismounting when faced with the prospect of assaulting a fortified position .

**greater middle east** 

**arabs** 

the islamic prophet muhammad made use of cavalry in many of his military campaigns including the expedition of dhu qarad ,and the expedition of zaid ibn haritha in al -is which took place in september ,627 ad ,fifth month of 6 ah of the islamic calendar .early organized arab mounted forces under the rashidun caliphate comprised a light cavalry armed with lance and sword .its main role was to attack the enemy flanks and rear
.these relatively lightly armored horsemen formed the most effective element of the muslim armies during the later stages of the islamic conquest of the levant .the best use of this lightly armed fast moving cavalry was revealed at the battle of yarmouk (636 ad )in which khalid ibn walid ,knowing the skills of his horsemen ,used them to turn the tables at every critical instance of the battle with their ability to engage ,disengage ,then turn back and attack again from the flank or rear .a strong cavalry regiment was formed by khalid ibn walid which included the veterans of the campaign of iraq and syria .early muslim historians have given it the name tali’a mutaharrikah (طليعة متحركة ),or the mobile guard .this was used as an advance guard and a strong striking force to route the opposing armies with its greater mobility that give it an upper hand when maneuvering against any byzantine army
.with this mobile striking force ,the conquest of syria was made easy
.the battle of talas in 751 ad was a conflict between the arab abbasid caliphate and the chinese tang dynasty over the control of central asia
.chinese infantry were routed by arab cavalry near the bank of the river talas . later mamluks were trained as cavalry soldiers .mamluks were to follow the dictates of al -furusiyya ,a code of conduct that included values like courage and generosity but also doctrine of cavalry tactics
,horsemanship ,archery and treatment of wounds .

**maghreb** 

the islamic berber states of north africa employed elite horse mounted cavalry armed with spears and following the model of the original arab occupiers of the region .horse -harness and weapons were manufactured locally and the six -monthly stipends for horsemen were double those of their infantry counterparts .during the 8th century islamic conquest of iberia large numbers of horses and riders were shipped from north africa
,to specialise in raiding and the provision of support for the massed berber footmen of the main armies .maghrebi traditions of mounted warfare eventually influenced a number of sub -saharan african polities in the medieval era .the esos of ikoyi ,military aristocrats of the yoruba peoples ,were a notable manifestation of this phenomenon .

**al -andalus** 

**iran** 

qizilbash ,were a class of safavid militant warriors in iran during the
15th to 18th centuries ,who often fought as elite cavalry .

**ottoman empire** 

during its period of greatest expansion ,from the 14th to 17th centuries
,cavalry formed the powerful core of the ottoman armies .registers dated
1475 record 22 ,000 sipahi feudal cavalry levied in europe ,17 ,000
sipahis recruited from anatolia ,and 3 ,000 kapikulu (regular body
-guard cavalry ).during the 18th century however the ottoman mounted troops evolved into light cavalry serving in the thinly populated regions of the middle east and north africa .such frontier horsemen were largely raised by local governors and were separate from the main field armies of the ottoman empire .at the beginning of the 19th century modernised nizam -i credit ("new army ")regiments appeared ,including full -time cavalry units officered from the horse guards of the sultan .

**renaissance europe** 

ironically ,the rise of infantry in the early 16th century coincided with the "golden age "of heavy cavalry ;a french or spanish army at the beginning of the century could have up to half its numbers made up of various kinds of light and heavy cavalry ,whereas in earlier medieval and later 17th -century armies the proportion of cavalry was seldom more than a quarter . knighthood largely lost its military functions and became more closely tied to social and economic prestige in an increasingly capitalistic western society .with the rise of drilled and trained infantry ,the mounted men -at -arms ,now sometimes called gendarmes and often part of the standing army themselves ,adopted the same role as in the hellenistic age ,that of delivering a decisive blow once the battle was already engaged ,either by charging the enemy in the flank or attacking their commander -in -chief .

from the 1550s onwards ,the use of gunpowder weapons solidified infantry's dominance of the battlefield and began to allow true mass armies to develop .this is closely related to the increase in the size of armies throughout the early modern period ;heavily armored cavalrymen were expensive to raise and maintain and it took years to train a skilled horseman or a horse ,while arquebusiers and later musketeers could be trained and kept in the field at much lower cost ,and were much easier to recruit . the spanish tercio and later formations relegated cavalry to a supporting role .the pistol was specifically developed to try to bring cavalry back into the conflict ,together with manoeuvres such as the caracole .the caracole was not particularly successful
,however ,and the charge (whether with lance ,sword ,or pistol )remained as the primary mode of employment for many types of european cavalry
,although by this time it was delivered in much deeper formations and with greater discipline than before .the demi -lancers and the heavily armored sword -and -pistol reiters were among the types of cavalry whose heyday was in the 16th and 17th centuries ,as for the polish winged hussars ,a heavy cavalry force that achieved great success against swedes ,russians ,and turks .

**18th -century europe and napoleonic wars** 

cavalry retained an important role in this age of regularization and standardization across european armies .they remained the primary choice for confronting enemy cavalry .attacking an unbroken infantry force head
-on usually resulted in failure ,but extended linear infantry formations were vulnerable to flank or rear attacks .cavalry was important at blenheim (1704 ),rossbach (1757 ),marengo (1800 ),eylau and friedland
(1807 ),remaining significant throughout the napoleonic wars . even with the increasing prominence of infantry ,cavalry still had an irreplaceable role in armies ,due to their greater mobility .their non
-battle duties often included patrolling the fringes of army encampments
,with standing orders to intercept suspected shirkers and deserters as well as : 257 , 266  serving as outpost pickets in advance of the main body .during battle ,lighter cavalry such as hussars and uhlans might skirmish with other cavalry ,attack light infantry ,or charge and either capture enemy artillery or render them useless by plugging the touchholes with iron spikes .heavier cavalry such as cuirassiers
,dragoons ,and carabiniers usually charged towards infantry formations or opposing cavalry in order to rout them .both light and heavy cavalry pursued retreating enemies ,the point where most battle casualties occurred .: 266 

the greatest cavalry charge of modern history was at the 1807 battle of eylau ,when the entire 11 ,000 -strong french cavalry reserve ,led by joachim murat ,launched a huge charge on and through the russian infantry lines .cavalry's dominating and menacing presence on the battlefield was countered by the use of infantry squares .the most notable examples are at the battle of quatre bras and later at the battle of waterloo ,the latter which the repeated charges by up to 9
,000 french cavalrymen ordered by michel ney failed to break the british
-allied army ,who had formed into squares .massed infantry ,especially those formed in squares were deadly to cavalry ,but offered an excellent target for artillery .once a bombardment had disordered the infantry formation ,cavalry were able to rout and pursue the scattered foot soldiers .it was not until individual firearms gained accuracy and improved rates of fire that cavalry was diminished in this role as well
.even then light cavalry remained an indispensable tool for scouting
,screening the army's movements ,and harassing the enemy's supply lines until military aircraft supplanted them in this role in the early stages of world war i .

**19th century** 

**europe** 

by the beginning of the 19th century ,european cavalry fell into four main categories :

cuirassiers ,heavy cavalry dragoons ,originally mounted infantry ,but later regarded as medium cavalry hussars ,light cavalry lancers or uhlans ,light cavalry ,primarily armed with lances there were cavalry variations for individual nations as well :france had the chasseurs à
cheval ;prussia had the jäger zu pferde ;bavaria ,saxony and austria had the chevaulegers ;and russia had cossacks .britain ,from the mid -18th century ,had light dragoons as light cavalry and dragoons ,dragoon guards and household cavalry as heavy cavalry .only after the end of the napoleonic wars were the household cavalry equipped with cuirasses ,and some other regiments were converted to lancers .in the united states army prior to 1862 the cavalry were almost always dragoons .the imperial japanese army had its cavalry uniformed as hussars ,but they fought as dragoons . in the crimean war ,the charge of the light brigade and the thin red line at the battle of balaclava showed the vulnerability of cavalry ,when deployed without effective support .

**franco -prussian war** 

during the franco -prussian war ,at the battle of mars -la -tour in 1870
,a prussian cavalry brigade decisively smashed the centre of the french battle line ,after skilfully concealing their approach .this event became known as von bredow's death ride after the brigade commander adalbert von bredow ;it would be used in the following decades to argue that massed cavalry charges still had a place on the modern battlefield
.

**imperial expansion** 

cavalry found a new role in colonial campaigns (irregular warfare
),where modern weapons were lacking and the slow moving infantry
-artillery train or fixed fortifications were often ineffective against indigenous insurgents (unless the latter offered a fight on an equal footing ,as at tel -el -kebir ,omdurman ,etc .).cavalry "flying columns
"proved effective ,or at least cost -effective ,in many campaigns—although an astute native commander (like samori in western africa ,shamil in the caucasus ,or any of the better boer commanders
)could turn the tables and use the greater mobility of their cavalry to offset their relative lack of firepower compared with european forces .
in 1903 the british indian army maintained forty regiments of cavalry
,numbering about 25 ,000 indian sowars (cavalrymen ),with british and indian officers . among the more famous regiments in the lineages of the modern indian and pakistani armies are :

governor general's bodyguard (now president's bodyguard ) skinner's horse (now india's 1st horse (skinner's horse )) gardner's lancers (now india's 2nd lancers (gardner's horse )) hodson's horse (now india's 3rd horse (hodson's ))of the bengal lancers fame 6th bengal cavalry (later amalgamated with 7th hariana lancers to form 18th king edward's own cavalry )now 18th cavalry of the indian army probyn's horse (now 5th horse ,pakistan ) royal deccan horse (now india's the deccan horse )
poona horse (now india's the poona horse ) scinde horse (now india's the scinde horse ) queen's own guides cavalry (now pakistan ). 11th prince albert victor's own cavalry (frontier force )(now 11th cavalry (frontier force ),pakistan )several of these formations are still active ,though they now are armoured formations ,for example the guides cavalry of pakistan .

the french army maintained substantial cavalry forces in algeria and morocco from 1830 until the end of the second world war .much of the mediterranean coastal terrain was suitable for mounted action and there was a long established culture of horsemanship amongst the arab and berber inhabitants .the french forces included spahis ,chasseurs d'
afrique ,foreign legion cavalry and mounted goumiers .both spain and italy raised cavalry regiments from amongst the indigenous horsemen of their north african territories (see regulares ,italian spahis and savari respectively ). imperial germany employed mounted formations in south west africa as part of the schutztruppen (colonial army
)garrisoning the territory .

**united states** 

in the early american civil war the regular united states army mounted rifle ,dragoon ,and two existing cavalry regiments were reorganized and renamed cavalry regiments ,of which there were six .over a hundred other federal and state cavalry regiments were organized ,but the infantry played a much larger role in many battles due to its larger numbers
,lower cost per rifle fielded ,and much easier recruitment .however
,cavalry saw a role as part of screening forces and in foraging and scouting .the later phases of the war saw the federal army developing a truly effective cavalry force fighting as scouts ,raiders ,and ,with repeating rifles ,as mounted infantry .the distinguished 1st virginia cavalry ranks as one of the most effectual and successful cavalry units on the confederate side .noted cavalry commanders included confederate general j .e .b .stuart ,nathan bedford forrest ,and john singleton mosby (a .k .a ."the grey ghost ")and on the union side ,philip sheridan and george armstrong custer . post civil war ,as the volunteer armies disbanded ,the regular army cavalry regiments increased in number from six to ten ,among them custer's u .s .7th cavalry regiment of little bighorn fame ,and the african -american u .s .9th cavalry regiment and u
.s .10th cavalry regiment .the black units ,along with others (both cavalry and infantry ),collectively became known as the buffalo soldiers
.according to robert m .utley :

the frontier army was a conventional military force trying to control
,by conventional military methods ,a people that did not behave like conventional enemies and ,indeed ,quite often were not enemies at all
.this is the most difficult of all military assignments ,whether in africa ,asia ,or the american west .these regiments ,which rarely took the field as complete organizations ,served throughout the american indian wars through the close of the frontier in the 1890s .volunteer cavalry regiments like the rough riders consisted of horsemen such as cowboys ,ranchers and other outdoorsmen ,that served as a cavalry in the united states military .

**first world war** 

**pre -war developments** 

at the beginning of the 20th century all armies still maintained substantial cavalry forces ,although there was contention over whether their role should revert to that of mounted infantry (the historic dragoon function ).following the experience of the south african war of
1899 –1902 (where mounted boer citizen commandos fighting on foot from cover proved more effective than regular cavalry )the british army withdrew lances for all but ceremonial purposes and placed a new emphasis on training for dismounted action in 1903 .an army order dated
1909 however instructed that the six british lancer regiments then in existence resume use of this impressive but obsolete weapon for active service .in 1882 the imperial russian army converted all its line hussar and lancer regiments to dragoons ,with an emphasis on mounted infantry training .in 1910 these regiments reverted to their historic roles
,designations and uniforms .by 1909 official regulations dictating the role of the imperial german cavalry had been revised to indicate an increasing realization of the realities of modern warfare .the massive cavalry charge in three waves which had previously marked the end of annual maneuvers was discontinued and a new emphasis was placed in training on scouting ,raiding and pursuit ;rather than main battle involvement .the perceived importance of cavalry was however still evident ,with thirteen new regiments of mounted rifles (jäger zu pferde
)being raised shortly before the outbreak of war in 1914 .in spite of significant experience in mounted warfare in morocco during 1908 –14
,the french cavalry remained a highly conservative institution .the traditional tactical distinctions between heavy ,medium ,and light cavalry branches were retained .french cuirassiers wore breastplates and plumed helmets unchanged from the napoleonic period ,during the early months of world war i .dragoons were similarly equipped ,though they did not wear cuirasses and did carry lances . light cavalry were described as being "a blaze of colour ".french cavalry of all branches were well mounted and were trained to change position and charge at full gallop
.one weakness in training was that french cavalrymen seldom dismounted on the march and their horses suffered heavily from raw backs in august
1914 .

**opening stages** 

**europe 1914** 

in august 1914 all combatant armies still retained substantial numbers of cavalry and the mobile nature of the opening battles on both eastern and western fronts provided a number of instances of traditional cavalry actions ,though on a smaller and more scattered scale than those of previous wars .the 110 regiments of imperial german cavalry ,while as colourful and traditional as any in peacetime appearance ,had adopted a practice of falling back on infantry support when any substantial opposition was encountered .these cautious tactics aroused derision amongst their more conservative french and russian opponents but proved appropriate to the new nature of warfare .a single attempt by the german army ,on 12 august 1914 ,to use six regiments of massed cavalry to cut off the belgian field army from antwerp foundered when they were driven back in disorder by rifle fire .the two german cavalry brigades involved lost 492 men and 843 horses in repeated charges against dismounted belgian lancers and infantry .one of the last recorded charges by french cavalry took place on the night of 9 /10 september 1914 when a squadron of the 16th dragoons overran a german airfield at soissons ,while suffering heavy losses . once the front lines stabilised on the western front with the start of trench warfare ,a combination of barbed wire
,uneven muddy terrain ,machine guns and rapid fire rifles proved deadly to horse mounted troops and by early 1915 most cavalry units were no longer seeing front line action . on the eastern front a more fluid form of warfare arose from flat open terrain favorable to mounted warfare .on the outbreak of war in 1914 the bulk of the russian cavalry was deployed at full strength in frontier garrisons and ,during the period that the main armies were mobilizing ,scouting and raiding into east prussia and austrian galicia was undertaken by mounted troops trained to fight with sabre and lance in the traditional style .on 21 august 1914 the 4th austro -hungarian kavalleriedivison fought a major mounted engagement at jaroslavic with the russian 10th cavalry division ,in what was arguably the final historic battle to involve thousands of horsemen on both sides
.while this was the last massed cavalry encounter on the eastern front
,the absence of good roads limited the use of mechanized transport and even the technologically advanced imperial german army continued to deploy up to twenty -four horse -mounted divisions in the east ,as late as 1917 .

**europe 1915 –18** 

for the remainder of the war on the western front cavalry had virtually no role to play .the british and french armies dismounted many of their cavalry regiments and used them in infantry and other roles :the life guards for example spent the last months of the war as a machine gun corps ;and the australian light horse served as light infantry during the gallipoli campaign .in september 1914 cavalry comprised 9 .28% of the total manpower of the british expeditionary force in france—by july
1918 this proportion had fallen to 1 .65% .as early as the first winter of the war most french cavalry regiments had dismounted a squadron each
,for service in the trenches .the french cavalry numbered 102 ,000 in may 1915 but had been reduced to 63 ,000 by october 1918 . the german army dismounted nearly all their cavalry in the west ,maintaining only one mounted division on that front by january 1917 .

italy entered the war in 1915 with thirty regiments of line cavalry
,lancers and light horse .while employed effectively against their austro -hungarian counterparts during the initial offensives across the isonzo river ,the italian mounted forces ceased to have a significant role as the front shifted into mountainous terrain .by 1916 most cavalry machine -gun sections and two complete cavalry divisions had been dismounted and seconded to the infantry .some cavalry were retained as mounted troops in reserve behind the lines ,in anticipation of a penetration of the opposing trenches that it seemed would never come
.tanks ,introduced on the western front by the british in september 1916
during the battle of the somme ,had the capacity to achieve such breakthroughs but did not have the reliable range to exploit them .in their first major use at the battle of cambrai (1917 ),the plan was for a cavalry division to follow behind the tanks ,however they were not able to cross a canal because a tank had broken the only bridge .on a few other occasions ,throughout the war ,cavalry were readied in significant numbers for involvement in major offensives ;such as in the battle of caporetto and the battle of moreuil wood .however it was not until the german army had been forced to retreat in the hundred days offensive of 1918 ,that limited numbers of cavalry were again able to operate with any effectiveness in their intended role .there was a successful charge by the british 7th dragoon guards on the last day of the war .in the wider spaces of the eastern front a more fluid form of warfare continued and there was still a use for mounted troops .some wide -ranging actions were fought ,again mostly in the early months of the war .however ,even here the value of cavalry was overrated and the maintenance of large mounted formations at the front by the russian army put a major strain on the railway system ,to little strategic advantage
.in february 1917 the russian regular cavalry (exclusive of cossacks
)was reduced by nearly a third from its peak number of 200 ,000 ,as two squadrons of each regiment were dismounted and incorporated into additional infantry battalions .their austro -hungarian opponents
,plagued by a shortage of trained infantry ,had been obliged to progressively convert most horse cavalry regiments to dismounted rifle units starting in late 1914 .

**middle east** 

in the middle east ,during the sinai and palestine campaign mounted forces (british ,indian ,ottoman ,australian ,arab and new zealand
)retained an important strategic role both as mounted infantry and cavalry . in egypt the mounted infantry formations like the new zealand mounted rifles brigade and australian light horse of anzac mounted division ,operating as mounted infantry ,drove german and ottoman forces back from romani to magdhaba and rafa and out of the egyptian sinai peninsula in 1916 . after a stalemate on the gaza—beersheba line between march and october 1917 ,beersheba was captured by the australian mounted division's 4th light horse brigade .their mounted charge succeeded after a coordinated attack by the british infantry and yeomanry cavalry and the australian and new zealand light horse and mounted rifles brigades
.a series of coordinated attacks by these egyptian expeditionary force infantry and mounted troops were also successful at the battle of mughar ridge ,during which the british infantry divisions and the desert mounted corps drove two ottoman armies back to the jaffa—jerusalem line
.the infantry with mainly dismounted cavalry and mounted infantry fought in the judean hills to eventually almost encircle jerusalem which was occupied shortly after . during a pause in operations necessitated by the german spring offensive in 1918 on the western front joint infantry and mounted infantry attacks towards amman and es salt resulted in retreats back to the jordan valley which continued to be occupied by mounted divisions during the summer of 1918 . the australian mounted division was armed with swords and in september ,after the successful breaching of the ottoman line on the mediterranean coast by the british empire infantry xxi corps was followed by cavalry attacks by the 4th cavalry division ,5th cavalry division and australian mounted divisions which almost encircled two ottoman armies in the judean hills forcing their retreat .meanwhile ,chaytor's force of infantry and mounted infantry in anzac mounted division held the jordan valley ,covering the right flank to later advance eastwards to capture es salt and amman and half of a third ottoman army .a subsequent pursuit by the 4th cavalry division and the australian mounted division followed by the 5th cavalry division to damascus .armoured cars and 5th cavalry division lancers were continuing the pursuit of ottoman units north of aleppo when the armistice of mudros was signed by the ottoman empire .

**post –world war i** 

a combination of military conservatism in almost all armies and post
-war financial constraints prevented the lessons of 1914 –1918 being acted on immediately .there was a general reduction in the number of cavalry regiments in the british ,french ,italian and other western armies but it was still argued with conviction (for example in the 1922
edition of the encyclopædia britannica )that mounted troops had a major role to play in future warfare .the 1920s saw an interim period during which cavalry remained as a proud and conspicuous element of all major armies ,though much less so than prior to 1914 . cavalry was extensively used in the russian civil war and the soviet -polish war .the last major cavalry battle was the battle of komarów in 1920 ,between poland and the russian bolsheviks .colonial warfare in morocco ,syria ,the middle east and the north west frontier of india provided some opportunities for mounted action against enemies lacking advanced weaponry .

the post -war german army (reichsheer )was permitted a large proportion of cavalry (18 regiments or 16 .4% of total manpower )under the conditions of the treaty of versailles .the british army mechanised all cavalry regiments between 1929 and 1941 ,redefining their role from horse to armoured vehicles to form the royal armoured corps together with the royal tank regiment .the u .s .cavalry abandoned its sabres in
1934 and commenced the conversion of its horsed regiments to mechanized cavalry ,starting with the first regiment of cavalry in january 1933 .
during the turkish war of independence turkish cavalry under general fahrettin altay was instrumental in kemalist victory over the invading greek army in 1922 ,during the battle of dumlupınar .v .cavalry division was able to slip behind the greek army ,cutting off all communication and supply lines as well as all retreat venues ,forcing the surrender of the remaining greek army which may have been the last time in history cavalry played a definitive role in the outcome of a battle . during the
1930s the french army experimented with integrating mounted and mechanised cavalry units into larger formations .dragoon regiments were converted to motorised infantry (trucks and motor cycles ),and cuirassiers to armoured units ;while light cavalry (chasseurs a' cheval
,hussars and spahis )remained as mounted sabre squadrons .the theory was that mixed forces comprising these diverse units could utilise the strengths of each according to circumstances .in practice mounted troops proved unable to keep up with fast moving mechanised units over any distance . the thirty -nine cavalry regiments of the british indian army were reduced to twenty -one as the result of a series of amalgamations immediately following world war i .the new establishment remained unchanged until 1936 when three regiments were redesignated as permanent training units ,each with six ,still mounted ,regiments linked to them
.in 1938 the process of mechanization began with the conversion of a full cavalry brigade (two indian regiments and one british )to armoured car and tank units .by the end of 1940 all of the indian cavalry had been mechanized ,initially and in the majority of cases ,to motorized infantry transported in 15cwt trucks .the last horsed regiment of the british indian army (other than the viceroy's bodyguard and some indian states forces regiments )was the 19th king george's own lancers which had its final mounted parade at rawalpindi on 28 october 1939 .this unit still exists in the pakistan army as an armored regiment .

**world war ii** 

while most armies still maintained cavalry units at the outbreak of world war ii in 1939 ,significant mounted action was largely restricted to the polish ,balkan ,and soviet campaigns .rather than charge their mounts into battle ,cavalry units were either used as mounted infantry
(using horses to move into position and then dismounting for combat )or as reconnaissance units (especially in areas not suited to tracked or wheeled vehicles ).

**polish** 

a popular myth is that polish cavalry armed with lances charged german tanks during the september 1939 campaign .this arose from misreporting of a single clash on 1 september near krojanty ,when two squadrons of the polish 18th lancers armed with sabres scattered german infantry before being caught in the open by german armoured cars . two examples illustrate how the myth developed .first ,because motorised vehicles were in short supply ,the poles used horses to pull anti -tank weapons into position .second ,there were a few incidents when polish cavalry was trapped by german tanks ,and attempted to fight free .however ,this did not mean that the polish army chose to attack tanks with horse cavalry .later ,on the eastern front ,the red army did deploy cavalry units effectively against the germans .

a more correct term would be "mounted infantry "instead of "cavalry ",as horses were primarily used as a means of transportation ,for which they were very suitable in view of the very poor road conditions in pre -war poland .another myth describes polish cavalry as being armed with both sabres and lances ;lances were used for peacetime ceremonial purposes only and the primary weapon of the polish cavalryman in 1939 was a rifle
.individual equipment did include a sabre ,probably because of well
-established tradition ,and in the case of a melee combat this secondary weapon would probably be more effective than a rifle and bayonet
.moreover ,the polish cavalry brigade order of battle in 1939 included
,apart from the mounted soldiers themselves ,light and heavy machine guns (wheeled ),the anti -tank rifle ,model 35 ,anti -aircraft weapons
,anti tank artillery such as the bofors 37 mm ,also light and scout tanks ,etc .the last cavalry vs .cavalry mutual charge in europe took place in poland during the battle of krasnobród ,when polish and german cavalry units clashed with each other . the last classical cavalry charge of the war took place on march 1 ,1945 ,during the battle of schoenfeld by the 1st "warsaw "independent cavalry brigade .infantry and tanks had been employed to little effect against the german position
,both of which floundered in the open wetlands only to be dominated by infantry and antitank fire from the german fortifications on the forward slope of hill 157 ,overlooking the wetlands .the germans had not taken cavalry into consideration when fortifying their position which
,combined with the "warsaw "s swift assault ,overran the german anti
-tank guns and consolidated into an attack into the village itself ,now supported by infantry and tanks .

**greek** 

the italian invasion of greece in october 1940 saw mounted cavalry used effectively by the greek defenders along the mountainous frontier with albania .three greek cavalry regiments (two mounted and one partially mechanized )played an important role in the italian defeat in this difficult terrain .

**soviet** 

the contribution of soviet cavalry to the development of modern military operational doctrine and its importance in defeating nazi germany has been eclipsed by the higher profile of tanks and airplanes .despite the view portrayed by german propaganda ,soviet cavalry contributed significantly to the defeat of the axis armies .their contributions included being the most mobile troops in the early stages ,when trucks and other equipment were low in quality ;as well as providing cover for retreating forces . considering their relatively limited numbers ,the soviet cavalry played a significant role in giving germany its first real defeats in the early stages of the war .the continuing potential of mounted troops was demonstrated during the battle of moscow ,against guderian and the powerful central german 9th army .cavalry were amongst the first soviet units to complete the encirclement in the battle of stalingrad ,thus sealing the fate of the german 6th army .mounted soviet forces also played a role in the encirclement of berlin ,with some cossack cavalry units reaching the reichstag in april 1945 .throughout the war they performed important tasks such as the capture of bridgeheads which is considered one of the hardest jobs in battle ,often doing so with inferior numbers .for instance the 8th guards cavalry regiment of the 2nd guards cavalry division ,often fought outnumbered against the best german units . by the final stages of the war only the soviet union was still fielding mounted units in substantial numbers
,some in combined mechanized and horse units .the advantage of this approach was that in exploitation mounted infantry could keep pace with advancing tanks .other factors favoring the retention of mounted forces included the high quality of russian cossacks which made about half of all cavalry ;and the relative lack of roads suitable for wheeled vehicles in many parts of the eastern front .another consideration was that the logistic capacity required to support very large motorized forces exceeded that necessary for mounted troops .the main usage of the soviet cavalry involved infiltration through front lines with subsequent deep raids ,which disorganized german supply lines .another role was the pursuit of retreating enemy forces during major frontline operations and breakthroughs .

**hungarian** 

during world war 2 the royal hungarian army's hussars were typically making only reconnaissance tasks against the soviets ,and only in a section or a squadron . however ,there was an incident documented by a german military officer .the last documentable hussar attack was conducted by lieutenant colonel kálmán mikecz on august 16 ,1941 ,at nikolaev .the hussars arrived as reinforcements ,they had to break through russian positions in front of german soldiers who ,according to their own confessions ,tried in vain to break through russian lines .the hussars equipped with swords and submachine guns broke through the front in a single attack .according to the accounts of astonished eyewitnesses
,they came into history as if they had run the red army with a sword .
eyewitness account of the last hussar attack : "… we were again in a tough fight with the desperately defensive enemy who dug himself along a high railway embankment .we’ve been attacked four times already ,and we’ve been kicked back all four times .the battalion commander swore
,but the company commanders were helpless .then ,instead of the artillery support we asked for countless times ,a hungarian hussar regiment appeared on the scene .we laughed .what the hell do they want here with their graceful ,elegant horses? we froze at once :these hungarians went crazy .cavalry squadron approached after a cavalry squdron .the command word rang .the bronze -brown ,slender riders almost grew to their saddle . their shining colonel of golden parolis jerked his sword .four or five armored cars cut out of the wings ,and the regiment slashed across the wide plain with flashing swords in the afternoon sun .seydlitz attacked like this once before .forgetting all caution ,we climbed out of our covers .it was all like a great equestrian movie .the first shots rumbled ,then became less frequent
.with astonished eyes ,in disbelief ,we watched as the soviet regiment
,which had so far repulsed our attacks with desperate determination ,now turned around and left its positions in panic .and the triumphant hungarians chased the russian in front of them and shredded them with their glittering sabers .the hussar sword ,it seems ,was a bit much for the nerves of russians .now ,for once ,the ancient weapon has triumphed over modern equipment ...."-erich kern ,a german officer ,wrote his memoir in 1948 .

**italian** 

the last mounted sabre charge by italian cavalry occurred on august 24
,1942 ,at isbuscenski (russia ),when a squadron of the savoia cavalry regiment charged the 812th siberian infantry regiment .the remainder of the regiment ,together with the novara lancers made a dismounted attack in an action that ended with the retreat of the russians after heavy losses on both sides .the final italian cavalry action occurred on october 17 ,1942 ,in poloj (now croatia )by a squadron of the alexandria cavalry regiment against a large group of yugoslav partisans .

**other axis** 

romanian ,hungarian and italian cavalry were dispersed or disbanded following the retreat of the axis forces from russia .germany still maintained some mounted (mixed with bicycles )ss and cossack units until the last days of the war .

**finnish** 

finland used mounted troops against russian forces effectively in forested terrain during the continuation war .the last finnish cavalry unit was not disbanded until 1947 .

**united states** 

the u .s .army's last horse cavalry actions were fought during world war ii :a )by the 26th cavalry regiment—a small mounted regiment of philippine scouts which fought the japanese during the retreat down the bataan peninsula ,until it was effectively destroyed by january 1942
;and b )on captured german horses by the mounted reconnaissance section of the u .s .10th mountain division in a spearhead pursuit of the german army across the po valley in italy in april 1945 .the last horsed u .s
.cavalry (the second cavalry division )were dismounted in march 1944 .

**british empire** 

all british army cavalry regiments had been mechanised since 1 march
1942 when the queen's own yorkshire dragoons (yeomanry )was converted to a motorised role ,following mounted service against the vichy french in syria the previous year .the final cavalry charge by british empire forces occurred on 21 march 1942 when a 60 strong patrol of the burma frontier force encountered japanese infantry near toungoo airfield in central myanmar .the sikh sowars of the frontier force cavalry ,led by captain arthur sandeman of the central india horse (21st king george v's own horse ),charged in the old style with sabres and most were killed .

**mongolia** 

in the early stages of world war ii ,mounted units of the mongolian people's army were involved in the battle of khalkhin gol against invading japanese forces .soviet forces under the command of georgy zhukov ,together with mongolian forces ,defeated the japanese sixth army and effectively ended the soviet –japanese border wars .after the soviet
–japanese neutrality pact of 1941 ,mongolia remained neutral throughout most of the war ,but its geographical situation meant that the country served as a buffer between japanese forces and the soviet union .in addition to keeping around 10% of the population under arms ,mongolia provided half a million trained horses for use by the soviet army .in
1945 a partially mounted soviet -mongolian cavalry mechanized group played a supporting role on the western flank of the soviet invasion of manchuria .the last active service seen by cavalry units of the mongolian army occurred in 1946 –1948 ,during border clashes between mongolia and the republic of china .

**post –world war ii to the present day** 

while most modern "cavalry "units have some historic connection with formerly mounted troops this is not always the case .the modern irish defence forces (df )includes a "cavalry corps "equipped with armoured cars and scorpion tracked combat reconnaissance vehicles .the df has never included horse cavalry since its establishment in 1922 (other than a small mounted escort of blue hussars drawn from the artillery corps when required for ceremonial occasions ).however ,the mystique of the cavalry is such that the name has been introduced for what was always a mechanised force . some engagements in late 20th and early 21st century guerrilla wars involved mounted troops ,particularly against partisan or guerrilla fighters in areas with poor transport infrastructure .such units were not used as cavalry but rather as mounted infantry .examples occurred in afghanistan ,portuguese africa and rhodesia .the french army used existing mounted squadrons of spahis to a limited extent for patrol work during the algerian war (1954 –62 ).the swiss army maintained a mounted dragoon regiment for combat purposes until 1973 .the portuguese army used horse mounted cavalry with some success in the wars of independence in angola and mozambique in the 1960s and 1970s .during the
1964 –79 rhodesian bush war the rhodesian army created an elite mounted infantry unit called grey's scouts to fight unconventional actions against the rebel forces of robert mugabe and joshua nkomo .the horse mounted infantry of the scouts were effective and reportedly feared by their opponents in the rebel african forces .in the 1978 to present afghan civil war period there have been several instances of horse mounted combat . central and south american armies maintained mounted cavalry for longer than those of asia ,europe ,or north america .the mexican army included a number of horse mounted cavalry regiments as late as the mid -1990s and the chilean army had five such regiments in
1983 as mounted mountain troops .the soviet army retained horse cavalry divisions until 1955 .at the dissolution of the soviet union in 1991
,there was still an independent horse mounted cavalry squadron in kyrgyzstan .

**operational horse cavalry** 

today the indian army's 61st cavalry is reported to be the largest existing horse -mounted cavalry unit still having operational potential
.it was raised in 1951 from the amalgamated state cavalry squadrons of gwalior ,jodhpur ,and mysore .while primarily utilised for ceremonial purposes ,the regiment can be deployed for internal security or police roles if required .the 61st cavalry and the president's body guard parade in full dress uniform in new delhi each year in what is probably the largest assembly of traditional cavalry still to be seen in the world .both the indian and the pakistani armies maintain armoured regiments with the titles of lancers or horse ,dating back to the 19th century . as of 2007 ,the chinese people's liberation army employed two battalions of horse -mounted border guards in xinjiang for border patrol purposes .pla mounted units last saw action during border clashes with vietnam in the 1970s and 1980s ,after which most cavalry units were disbanded as part of major military downsizing in the 1980s .in the wake of the 2008 sichuan earthquake ,there were calls to rebuild the army horse inventory for disaster relief in difficult terrain .subsequent chinese media reports confirm that the pla maintains operational horse cavalry at squadron strength in xinjiang and inner mongolia for scouting
,logistical ,and border security purposes . the chilean army still maintains a mixed armoured cavalry regiment ,with elements of it acting as mounted mountain exploration troops ,based in the city of angol
,being part of the iii mountain division ,and another independent exploration cavalry detachment in the town of chaitén .the rugged mountain terrain calls for the use of special horses suited for that use
. the argentine army has two mounted cavalry units :the regiment of horse grenadiers ,which performs mostly ceremonial duties but at the same time is responsible for the president's security (in this case
,acting as infantry ),and the 4th mountain cavalry regiment (which comprises both horse and light armoured squadrons ),stationed in san martín de los andes ,where it has an exploration role as part the 6th mountain brigade .most armoured cavalry units of the army are considered successors to the old cavalry regiments from the independence wars ,and keep their traditional names ,such as hussars ,cuirassiers ,lancers ,etc
.,and uniforms .equestrian training remains an important part of their tradition ,especially among officers .

**ceremonial horse cavalry and armored cavalry retaining traditional titles** 

cavalry or mounted gendarmerie units continue to be maintained for purely or primarily ceremonial purposes by the algerian ,argentine
,bolivian ,brazilian ,british ,bulgarian ,canadian ,chilean ,colombian
,danish ,dutch ,finnish ,french ,hungarian ,indian ,italian ,jordanian
,malaysian ,moroccan ,nepalese ,nigerian ,north korean ,omani ,pakistani
,panamanian ,paraguayan ,peruvian ,polish ,portuguese ,russian
,senegalese ,spanish ,swedish ,thai ,tunisian ,turkmenistan ,united states ,uruguayan and venezuelan armed forces . a number of armoured regiments in the british army retain the historic designations of hussars ,dragoons ,light dragoons ,dragoon guards ,lancers and yeomanry
.only the household cavalry (consisting of the life guards' mounted squadron ,the blues and royals' mounted squadron ,the state trumpeters of the household cavalry and the household cavalry mounted band )are maintained for mounted (and dismounted )ceremonial duties in london .
the french army still has regiments with the historic designations of cuirassiers ,hussars ,chasseurs ,dragoons and spahis .only the cavalry of the republican guard and a ceremonial fanfare detachment of trumpeters for the cavalry /armoured branch as a whole are now mounted .
in the canadian army ,a number of regular and reserve units have cavalry roots ,including the royal canadian hussars (montreal ),the governor general's horse guards ,lord strathcona's horse ,the british columbia dragoons ,the royal canadian dragoons ,and the south alberta light horse
.of these ,only lord strathcona's horse and the governor general's horse guards maintain an official ceremonial horse -mounted cavalry troop or squadron .the modern pakistan army maintains about 40 armoured regiments with the historic titles of lancers ,cavalry or horse .six of these date back to the 19th century ,although only the president's body guard remains horse -mounted . in 2002 the army of the russian federation reintroduced a ceremonial mounted squadron wearing historic uniforms .
both the australian and new zealand armies follow the british practice of maintaining traditional titles (light horse or mounted rifles )for modern mechanised units .however ,neither country retains a horse
-mounted unit . several armored units of the modern united states army retain the designation of "armored cavalry ".the united states also has
"air cavalry "units equipped with helicopters .the horse cavalry detachment of the u .s .army's 1st cavalry division ,made up of active duty soldiers ,still functions as an active unit ,trained to approximate the weapons ,tools ,equipment and techniques used by the united states cavalry in the 1880s .

**non -combat support roles** 

the first troop philadelphia city cavalry is a volunteer unit within the pennsylvania army national guard which serves as a combat force when in federal service but acts in a mounted disaster relief role when in state service .in addition ,the parsons' mounted cavalry is a reserve officer training corps unit which forms part of the corps of cadets at texas a&m university .valley forge military academy and college also has a mounted company ,known as d -troop . some individual u .s .states maintain cavalry units as a part of their respective state defense forces .the maryland defense force includes a cavalry unit ,cavalry troop a ,which serves primarily as a ceremonial unit .the unit training includes a saber qualification course based upon the 1926 u .s .army course
.cavalry troop a also assists other maryland agencies as a rural search and rescue asset .in massachusetts ,the national lancers trace their lineage to a volunteer cavalry militia unit established in 1836 and are currently organized as an official part of the massachusetts organized militia .the national lancers maintain three units ,troops a ,b ,and c
,which serve in a ceremonial role and assist in search and rescue missions .in july 2004 ,the national lancers were ordered into active state service to guard camp curtis guild during the 2004 democratic national convention .the governor's horse guard of connecticut maintains two companies which are trained in urban crowd control .in 2020 ,the california state guard stood up the 26th mounted operations detachment
,a search -and -rescue cavalry unit .

**social status** 

from the beginning of civilization to the 20th century ,ownership of heavy cavalry horses has been a mark of wealth amongst settled peoples
.a cavalry horse involves considerable expense in breeding ,training
,feeding ,and equipment ,and has very little productive use except as a mode of transport . for this reason ,and because of their often decisive military role ,the cavalry has typically been associated with high social status .this was most clearly seen in the feudal system ,where a lord was expected to enter combat armored and on horseback and bring with him an entourage of lightly armed peasants on foot .if landlords and peasant levies came into conflict ,the poorly trained footmen would be ill -equipped to defeat armored knights . in later national armies
,service as an officer in the cavalry was generally a badge of high social status .for instance prior to 1914 most officers of british cavalry regiments came from a socially privileged background and the considerable expenses associated with their role generally required private means ,even after it became possible for officers of the line infantry regiments to live on their pay .options open to poorer cavalry officers in the various european armies included service with less fashionable (though often highly professional )frontier or colonial units .these included the british indian cavalry ,the russian cossacks or the french chasseurs d'afrique . during the 19th and early 20th centuries most monarchies maintained a mounted cavalry element in their royal or imperial guards .these ranged from small units providing ceremonial escorts and palace guards ,through to large formations intended for active service .the mounted escort of the spanish royal household provided an example of the former and the twelve cavalry regiments of the prussian imperial guard an example of the latter .in either case the officers of such units were likely to be drawn from the aristocracies of their respective societies .

**on film** 

some sense of the noise and power of a cavalry charge can be gained from the 1970 film waterloo ,which featured some 2 ,000 cavalrymen ,some of them cossacks .it included detailed displays of the horsemanship required to manage animal and weapons in large numbers at the gallop
(unlike the real battle of waterloo ,where deep mud significantly slowed the horses ).the gary cooper movie they came to cordura contains a scene of a cavalry regiment deploying from march to battle line formation .a smaller -scale cavalry charge can be seen in the lord of the rings :the return of the king (2003 );although the finished scene has substantial computer -generated imagery ,raw footage and reactions of the riders are shown in the extended version dvd appendices . other films that show cavalry actions include :

the charge of the light brigade ,about the battle of balaclava in the crimean war 40 ,000 horsemen ,about the australian light horse during the sinai and palestine campaign of world war i the lighthorsemen ,about the battle of beersheba ,1917 war horse ,about the british cavalry in europe during world war i hubal ,about the last months (september 1939
–april 1940 )of poland's first world war ii guerrilla ,major henryk dobrzański ,"hubal " the patriot includes light cavalry usage . and quiet flows the don depicts don cossacks during world war i kingdom of heaven includes a cavalry charge during the siege of kerak

**examples** 

**types** 

**units** 

**notable horse cavalrymen** 

georgios stanotas ,commander of the hellenic army's cavalry division during world war ii didier courrèges ,major in the french army ,member of école nationale d'équitation's cadre noir ,olympian at 2004 summer olympics edwin ramsey ,lieutenant colonel in the 26th cavalry regiment during world war ii ,recipient of the distinguished service cross ,led the last cavalry charge in american military history general fahrettin altay ,commander of the v .cavalry division of the turkish 1st army during the turkish war of independence ,which was instrumental in victory over the invading greek army .his name is given to the new turkish battle tank altay .atatürk'ün bütün eserleri ,cilt 27 ,kaynak yayınları ,1998 ,isbn 978 -975 -343 -235 -1 ,p .81 .

**gallery** 

**see also** 

cavalry tactics shock tactics horses in warfare armored reconnaissance
–a modern role in most militaries for 'cavalry' titled units

**notes** 

**references** 

ebrey ,walthall ,palais (2006 ).east asia :a cultural ,social ,and political history .boston :houghton mifflin company . :cs1 maint :uses authors parameter (link ) ebrey ,patricia buckley (1999 ).the cambridge illustrated history of china .cambridge :cambridge university press
.isbn 0 -521 -43519 -6 . falls ,cyril ;g .macmunn (1930 ).military operations egypt & palestine from the outbreak of war with germany to june 1917 .official history of the great war based on official documents by direction of the historical section of the committee of imperial defence .vol .1 .london :hm stationery office .oclc 610273484 . falls
,cyril ;a .f .becke (maps )(1930 ).military operations egypt & palestine from june 1917 to the end of the war .official history of the great war based on official documents by direction of the historical section of the committee of imperial defence .vol .2 part i .london :hm stationery office .oclc 644354483 . falls ,cyril ;a .f .becke (maps )(1930
).military operations egypt & palestine from june 1917 to the end of the war .official history of the great war based on official documents by direction of the historical section of the committee of imperial defence
.vol .2 part ii .london :hm stationery office .oclc 256950972 . lynn
,john albert ,giant of the grand siècle :the french army ,1610 –1715
,cambridge university press ,1997 menon ,shanti (april 1995 )."chariot racers of the steppes ".discover .archived from the original on 2007 -10
-13 . muir ,william (1883 ).annals of the early caliphate :from original sources .london :smith ,elder & co . needham ,joseph (1986 ).science and civilization in china .vol .4 ,physics and physical technology ,part 2
,mechanical engineering .taipei :caves books ,ltd . pargiter ,frederick eden ,dr .,chronology based on :ancient indian historical tradition
,oxford university press ,h .milford ,1924 ,reprint 1997 peers ,c .j
.(2006 ).soldiers of the dragon :chinese armies 1500 bc –ad 1840 .oxford
:osprey publishing . rodger ,n .a .m .(1999 ).the safeguard of the sea
:a naval history of britain 660 –1649 .w .w .norton & co ltd .isbn 0
-393 -04579 -x .

**external links** 

cavalryscouts .org napoleonic cavalry cavalry tactics from francis j
.lippitt's ,a treatise on the tactical use of the three arms :infantry
,artillery and cavalry (1865 ) cavalry in mass (u .s .report on russian cavalry organization and operations in world war ii ) society of the military horse gesellschaft der freunde der kavallerie (german ) the horse and mule in the british army during ww1 historic films showing cavalry during world war i at europeanfilmgateway .eu


# Answers/Give up?

In [8]:
#Give up? Uncomment(remove the # from) the line below and run 
print(name)

Cavalry
